In [8]:
# 删除road实体和关系
cypher = 'match(p:road) detach delete p'
g.run(cypher)

(No data)

In [9]:
# 删除所有 'touch' 关系
g.run("MATCH ()-[r:touch]->() DELETE r")

(No data)

In [10]:
cypher = 'MATCH (n:road) RETURN count(n) AS total_road_nodes'
g.run(cypher)

total_road_nodes
0


In [11]:
cypher = 'MATCH ()-[r:touch]->() RETURN count(r) AS touch_relation_count'
g.run(cypher)

touch_relation_count
0


In [1]:
global_var = 100000000000

In [2]:
import geopandas as gpd
from shapely import geometry
import matplotlib.pyplot as plt
import pandas as pd

road = r"F:/paper2/result/split_result.shp"
road = gpd.read_file(road)
# honey = r"F:\paper2\result\dz_honeycomb_125.shp"
# honey = gpd.read_file(honey)

# fig, ax = plt.subplots(figsize=(12, 8))
# ax = road.geometry.plot(ax=ax,fc='none',ec='k')
# ax = honey.geometry.plot(ax=ax)

PROJ: proj_create_from_database: Cannot find proj.db


In [3]:
#显示所有列
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',200)

In [4]:
print(road.crs)

PROJCS["CGCS2000_3-degree_Gauss-Kruger_CM_120E",GEOGCS["GCS_China_Geodetic_Coordinate_System_2000",DATUM["D_China_2000",SPHEROID["CGCS2000",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",120],PARAMETER["scale_factor",1],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [5]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher, Subgraph
import os
import json

In [6]:
g = Graph('bolt://localhost:7687',auth=('neo4j','your password'), name = 'trkg')

In [7]:
road

,pre_code,fclass,name,layer,oneway,bridge,tunnel,fclass中,描述,分类,index,distance,geometry
0,38377,primary,None,0.0,F,F,F,主要道路,城市主要车行道路,城市主干路,0,216.622015,"LINESTRING (611188.945 3408186.854, 611405.451 3408179.776)"
1,8080,primary,None,0.0,F,F,F,主要道路,城市主要车行道路,城市主干路,1,216.625870,"LINESTRING (611405.451 3408191.808, 611188.945 3408199.003)"
2,38377,primary,None,0.0,F,F,F,主要道路,城市主要车行道路,城市主干路,2,216.622015,"LINESTRING (610972.438 3408193.932, 611188.945 3408186.854)"
3,8080,primary,None,0.0,F,F,F,主要道路,城市主要车行道路,城市主干路,3,216.625870,"LINESTRING (611188.945 3408199.003, 610972.438 3408206.198)"
4,38377,primary,None,0.0,F,F,F,主要道路,城市主要车行道路,城市主干路,4,216.622015,"LINESTRING (609889.906 3408229.322, 610106.413 3408222.244)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110541,29747,secondary,None,0.0,B,F,F,次要道路,城市次要车行道路，机场外围车行道路,城市主干路,110541,217.761116,"LINESTRING (621797.756 3524764.073, 622014.262 3524740.730)"
110542,29747,secondary,None,0.0,B,F,F,次要道路,城市次要车行道路，机场外围车行道路,城市主干路,110542,217.867493,"LINESTRING (621364.743 3524812.515, 621581.249 3524788.199)"
110543,29747,secondary,None,0.0,B,F,F,次要道路,城市次要车行道路，机场外围车行道路,城市主干路,110543,217.846776,"LINESTRING (621581.249 3524788.199, 621755.590 3524768.619, 621797.756 3524764.073)"
110544,29747,secondary,None,0.0,B,F,F,次要道路,城市次要车行道路，机场外围车行道路,城市主干路,110544,198.842111,"LINESTRING (622014.262 3524740.730, 622211.958 3524719.415)"


In [12]:
#创建路网实体
road_ls = []
for i in range(len(road)):
    if i==global_var:
        break
    # print('road:', i)
    polyline = road.iloc[i]
    
    node = Node('road', name=int(polyline['index']), road_pcode=int(polyline['pre_code']), fclass=polyline['fclass'], cclass=polyline['分类'],
                roadname=polyline['name'], layer=int(polyline['layer']), bridge=polyline['bridge'], tunnel=polyline['tunnel'],
                discribe=polyline['描述'], distance=float(polyline['distance']), oneway=polyline['oneway'],
                geometry=str(polyline['geometry']))
    road_ls.append(node)

subgraph = Subgraph(nodes=road_ls)
tx = g.begin() 
tx.create(subgraph)
g.commit(tx)

In [13]:
# roadbuffer = r"F:\paper2\result\roadbuffer_125.shp"
# roadbuffer = gpd.read_file(roadbuffer)
# roadbuffer

In [14]:
node_matcher = NodeMatcher(g)
relation_matcher = RelationshipMatcher(g)

In [15]:
from tqdm.notebook import tqdm

# 1. 建立R树索引
spatial_index = road.sindex

# 2. 逐个处理道路节点
for i in tqdm(range(len(road)), total=len(road)):
    if i >= global_var:
        break
    
    polygon = road.loc[i, 'geometry']

    # 通过R树进行简单包含查询
    possible_matches_index = list(spatial_index.intersection(polygon.bounds))
    possible_matches = road.iloc[possible_matches_index]

    # 将简单查询结果进行匹配，得到精确查询结果
    precise_matches_intersects = possible_matches[possible_matches.touches(polygon)]
    
    k = precise_matches_intersects['index'].tolist()

    # 匹配当前处理的主道路节点
    main_road = node_matcher.match('road', name=i).first()

    # 处理与当前道路相邻的道路节点
    for j in k:
        if j >= global_var:
            break

        sub_road = node_matcher.match('road', name=j).first()

        # 确保关系方向一致：从 name 较小的节点指向 name 较大的节点
        if main_road['name'] < sub_road['name']:
            
            # 使用Cypher的MERGE语句保证关系的唯一性
            query = """
            MATCH (a:road {name: $main}), (b:road {name: $sub})
            MERGE (a)-[r:touch]-(b)
            RETURN r
            """
            g.run(query, main=main_road['name'], sub=sub_road['name'])

# 此时不再需要批量提交关系，Cypher中的MERGE操作已经处理了唯一性问题


  0%|          | 0/110546 [00:00<?, ?it/s]

In [ ]:
#道路与道路关系
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

def process_road(i):
    if i >= global_var:
        return []
    # print('road:', i)
    polygon = road.loc[i, 'geometry']

    # 通过R树进行简单包含查询
    possible_matches_index = list(spatial_index.intersection(polygon.bounds))
    possible_matches = road.iloc[possible_matches_index]

    # 将简单查询结果进行匹配，得到精确查询结果
    precise_matches_intersects = possible_matches[possible_matches.touches(polygon)]
    # print('precise_matches_intersects\n',precise_matches_intersects)
    
    # 排除自身，避免自相交
    # intersect_gdf = precise_matches_intersects[precise_matches_intersects['index'] != i]
    # print('intersect_gdf\n',intersect_gdf)
    
    k = precise_matches_intersects['index'].tolist()

    # 将拓扑关系添加进知识图谱
    main_road = node_matcher.match('road', name=i).first()
    
    relations = []
    for j in k:
        if j >= global_var:
            continue

        sub_road = node_matcher.match('road', name=j).first()

        # 确保关系方向一致：只从 name 较小的节点指向 name 较大的节点
        if main_road['name'] < sub_road['name']:

            # 使用Cypher查询检查两个节点之间是否存在"touch" 关系
            query = """
            MATCH (a:road {name: $main})-[r:touch]-(b:road {name: $sub})
            RETURN count(r) > 0 as relation_exists
            """
            relation_exists = g.run(query, main=main_road['name'], sub=sub_road['name']).evaluate()

            if not relation_exists:
            
                # 创建关系
                relation = Relationship(main_road, "touch", sub_road)
                relations.append(relation)

    return relations

relation_ls = []
spatial_index = road.sindex  # 建立R树索引

# 使用ThreadPoolExecutor进行并行处理，并加上进度条
with ThreadPoolExecutor(max_workers=120) as executor:
    for relations in tqdm(executor.map(process_road, range(len(road))), total=len(road)):
        relation_ls.extend(relations)

# 批量提交所有关系到数据库
subgraph = Subgraph(relationships=relation_ls)
tx = g.begin()
tx.create(subgraph)
g.commit(tx)
relation_ls.clear()


In [15]:
def print_touch_relation(main_road_name, sub_road_name, g):
    # 使用Cypher查询两个节点之间的 touch 关系
    query = """
    MATCH (a:road {name: $main})-[r:touch]-(b:road {name: $sub})
    RETURN r
    """
    
    # 执行查询并获取结果
    result = g.run(query, main=main_road_name, sub=sub_road_name).data()

    # 打印关系
    if result:
        for record in result:
            relation = record['r']
            print("Relationship found: ", relation)
    else:
        print(f"No touch relationship found between road {main_road_name} and road {sub_road_name}")

In [16]:

# 打印关系
print_touch_relation(1, 3, g)  # 例如，打印 road 1 和 road 2 之间的 touch 关系


Relationship found:  (1)-[:touch {}]->(3)
